## importing neccasry library 

In [67]:
# importing neccary  libary
from nsepy import get_history
import numpy as np
import pandas as pd 
from datetime import date 
import matplotlib.pyplot as plt 
%matplotlib inline 
from statsmodels.tsa.stattools import OLS 
from statsmodels.tsa.stattools import adfuller
import seaborn as sns

##  craeting sector list with thier respective sectors stocks selection of stocks was made based on volume 

In [9]:
# craeter sector wise variables with stocks list in it 
banking = ['ALBK','ANDHRABANK','AXISBANK','BANKBARODA','BANKINDIA','CANBK','FEDERALBNK','HDFC','ICICIBANK','IDBI','INDIANB','KOTAKBANK','ORIENTBANK','PNB','SBIN','SYNDIBANK','UNIONBANK']
it = ['INFY',"TCS","WIPRO","HCLTECH","TECHM","HEXAWARE"]
cement = ["INDIACEM","ULTRACEMCO","AMBUJACEM","ACC","CENTURYTEX"]
petroleum = ["RIIL","ONGC","GAIL","IOC","BPCL","HINDPETRO","PETRONET"]
auto = ["ESCORTS","M&M","TATAMOTORS","ASHOKLEY","BAJAJ-AUTO","HEROMOTOCO","TVSMOTOR"]
health = ["AUROPHARMA","SUNPHARMA","WOCKPHARMA","CADILAHC","PEL","BIOCON","LUPIN","GLENMARK","CIPLA"]
finance = ["RELCAPITAL","DHFL","IBULHSGFIN","MANAPPURAM","L&TFH","IFCI","RECLTD","PFC","HDFC"]

In [10]:
all_sectors_stocks = [banking,it,cement,petroleum,auto,health,finance]# storing all sectors in one list 

## craeting a user defined function wich calauclates rollign adf ,rolling beta,and  mean of that rolling adf 

In [11]:
def rolling_adf(x_name,y_name,start_date,end_date):# craeting a function wich calculates rollign adf 
    rolling_beta = []
    adf_values = []
    x = get_history(x_name,start = start_date,end=end_date)['Close']
    y = get_history(y_name,start = start_date,end=end_date)['Close']
    day_count = 0
    iteration_count = -1
    for i in x:                     #x is just birng used as a range
        day_count = day_count + 1    # day count turn to one in the first iteration and so on 
        if day_count >= 60:
            iteration_count = iteration_count + 1 #iteration starts adding up once day count turn 60 
            model = OLS(y[iteration_count:day_count].values,x[iteration_count:day_count].values).fit()
            spread = y[iteration_count:day_count] - (x[iteration_count:day_count] * model.params[0])
            adf = adfuller(spread)[0]
            rolling_beta.append(model.params[0])
            adf_values.append(adf)
    rolling_adf_mean = np.mean(adf_values)
    rolling_values = [adf_values,rolling_beta,rolling_adf_mean]
    return rolling_values

## banking sector test for year 2015

In [12]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [13]:
banking_pairs= [(x,y) for x in banking for y in banking if x!=y] #craeting combination of stocks so the are iterable 
banking_dict = {}
for x,y in banking_pairs: #running the function on all bankin pairs we craeted and storing thier values in the dict 
    banking_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [14]:
#fetching pairs banking dict wich have rolling adf mean below -2.5
banking_tradable_pairs = []
banking_tradable_stocks = []
for x,y in banking_pairs:
    mean = banking_dict[str(x) + str(y)][2]
    if mean <= -2.5:                        
        banking_tradable_pairs.append((x,y))
        banking_tradable_stocks.append(x)
        banking_tradable_stocks.append(y)
banking_tradable_pairs = set(tuple(sorted(p)) for p in banking_tradable_pairs)# removing the duplicates

In [15]:
print(set(banking_tradable_stocks))# i need this so i could copy this to my blue shift algo 
print(banking_tradable_pairs)

{'AXISBANK', 'SBIN', 'INDIANB', 'FEDERALBNK', 'BANKBARODA', 'BANKINDIA', 'ICICIBANK', 'HDFC', 'ANDHRABANK', 'UNIONBANK', 'ALBK'}
{('AXISBANK', 'FEDERALBNK'), ('FEDERALBNK', 'ICICIBANK'), ('ICICIBANK', 'INDIANB'), ('AXISBANK', 'INDIANB'), ('AXISBANK', 'BANKINDIA'), ('HDFC', 'ICICIBANK'), ('ALBK', 'SBIN'), ('ANDHRABANK', 'SBIN'), ('ANDHRABANK', 'INDIANB'), ('BANKBARODA', 'FEDERALBNK'), ('FEDERALBNK', 'UNIONBANK')}


In [227]:
def dataframe(sector_dict,dict_pairs):
    rolling_adf_pvalue_mean = []
    rolling_beta_value = []
    x_stocks = []
    y_stocks = []
    for x,y in dict_pairs:
        adf_mean = sector_dict[str(x) + str(y)][2]
        rolling_beta_mean = np.mean(sector_dict[str(x) + str(y)][1])
        rolling_adf_pvalue_mean.append(adf_mean)
        x_stocks.append(x)
        y_stocks.append(y)
        rolling_beta_value.append(rolling_beta_mean)
    
    df = pd.DataFrame({"x_stock":x_stocks,"y_stock":y_stocks,
                                 "rolling_adf_mean":rolling_adf_pvalue_mean,
                                 "rolling_beta_mean":rolling_beta_value})
    return df


In [228]:
dataframe(banking_dict,banking_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,AXISBANK,FEDERALBNK,-2.682124,0.132610
1,FEDERALBNK,ICICIBANK,-2.261078,4.089730
2,ICICIBANK,INDIANB,-2.649305,0.475056
3,AXISBANK,INDIANB,-2.550179,0.261110
4,AXISBANK,BANKINDIA,-2.497130,0.286539
5,HDFC,ICICIBANK,-2.597397,0.228822
6,ALBK,SBIN,-2.520664,3.067610
7,ANDHRABANK,SBIN,-2.694244,3.660726
8,ANDHRABANK,INDIANB,-2.735978,1.971406
9,BANKBARODA,FEDERALBNK,-2.635142,0.391012


## it sector test 2015 

In [17]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [18]:
it_pairs= [(x,y) for x in it for y in it if x!=y] #craeting combination of stocks so the are iterable 
it_dict = {}

In [19]:
for x,y in it_pairs:
    it_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [20]:
it_tradable_pairs = []
it_tradable_stocks = []
for x,y in it_pairs:
    mean = it_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        it_tradable_pairs.append((x,y))
        it_tradable_stocks.append(x)
        it_tradable_stocks.append(y)
it_tradable_pairs = set(tuple(sorted(p)) for p in it_tradable_pairs)

In [21]:
print(set(it_tradable_stocks))
print(it_tradable_pairs)

{'HCLTECH', 'INFY', 'WIPRO', 'TECHM'}
{('INFY', 'TECHM'), ('HCLTECH', 'WIPRO'), ('INFY', 'WIPRO'), ('TECHM', 'WIPRO')}


In [226]:
dataframe(it_dict,it_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,INFY,TECHM,-3.181709,0.483666
1,HCLTECH,WIPRO,-2.561448,0.623974
2,INFY,WIPRO,-2.531546,0.514278
3,TECHM,WIPRO,-4.364694,1.063505


## cement sector test 2015

In [22]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [23]:
cement_pairs= [(x,y) for x in cement for y in cement if x!=y] #craeting combination of stocks so the are iterable 
cement_dict = {}

In [24]:
for x,y in cement_pairs:
    cement_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [25]:
cement_tradable_pairs = []
cement_tradable_stocks = []
for x,y in cement_pairs:
    mean = cement_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        cement_tradable_pairs.append((x,y))
        cement_tradable_stocks.append(x)
        cement_tradable_stocks.append(y)
cement_tradable_pairs = set(tuple(sorted(p)) for p in cement_tradable_pairs)

In [26]:
print(set(cement_tradable_stocks))
print(cement_tradable_pairs)

set()
set()


In [219]:
dataframe(cement_dict,cement_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean


##  petroleum sector test 2015

In [27]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [28]:
petroleum_pairs= [(x,y) for x in petroleum for y in petroleum if x!=y] #craeting combination of stocks so the are iterable 
petroleum_dict = {}

In [29]:
for x,y in petroleum_pairs:
    petroleum_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [30]:
petroleum_tradable_pairs = []
petroleum_tradable_stocks = []
for x,y in petroleum_pairs:
    mean = petroleum_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        petroleum_tradable_pairs.append((x,y))
        petroleum_tradable_stocks.append(x)
        petroleum_tradable_stocks.append(y)
petroleum_tradable_pairs = set(tuple(sorted(p)) for p in petroleum_tradable_pairs)

In [31]:
print(set(petroleum_tradable_stocks))
print(petroleum_tradable_pairs)

{'RIIL', 'ONGC', 'GAIL'}
{('GAIL', 'RIIL'), ('GAIL', 'ONGC')}


In [221]:
dataframe(petroleum_dict,petroleum_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,GAIL,RIIL,-2.915664,1.066313
1,GAIL,ONGC,-2.896549,0.782810


## auto sector test 2015

In [32]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [33]:
auto_pairs= [(x,y) for x in auto for y in auto if x!=y] #craeting combination of stocks so the are iterable 
auto_dict = {}

In [34]:
for x,y in auto_pairs:
    auto_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [35]:
auto_tradable_pairs = []
auto_tradable_stocks = []
for x,y in auto_pairs:
    mean = auto_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        auto_tradable_pairs.append((x,y))
        auto_tradable_stocks.append(x)
        auto_tradable_stocks.append(y)
auto_tradable_pairs = set(tuple(sorted(p)) for p in auto_tradable_pairs)

In [36]:
print(set(auto_tradable_stocks))
print(auto_tradable_pairs)

{'HEROMOTOCO', 'ASHOKLEY', 'ESCORTS', 'BAJAJ-AUTO'}
{('ASHOKLEY', 'ESCORTS'), ('BAJAJ-AUTO', 'HEROMOTOCO')}


In [222]:
dataframe(auto_dict,auto_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,ASHOKLEY,ESCORTS,-2.500983,1.778373
1,BAJAJ-AUTO,HEROMOTOCO,-3.114746,1.049928


## health sector test 2015

In [37]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [38]:
health_pairs= [(x,y) for x in health for y in health if x!=y] #craeting combination of stocks so the are iterable 
health_dict = {}

In [39]:
for x,y in health_pairs:
    health_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [40]:
health_tradable_pairs = []
health_tradable_stocks = []
for x,y in health_pairs:
    mean = health_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        health_tradable_pairs.append((x,y))
        health_tradable_stocks.append(x)
        health_tradable_stocks.append(y)
health_tradable_pairs = set(tuple(sorted(p)) for p in health_tradable_pairs)

In [41]:
print(set(health_tradable_stocks))
print(health_tradable_pairs)

{'CADILAHC', 'PEL', 'AUROPHARMA'}
{('CADILAHC', 'PEL'), ('AUROPHARMA', 'CADILAHC'), ('AUROPHARMA', 'PEL')}


In [224]:
dataframe(health_dict,health_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,CADILAHC,PEL,-2.511376,0.571094
1,AUROPHARMA,CADILAHC,-3.283342,1.569355
2,AUROPHARMA,PEL,-3.368600,1.069800


## finance sector test for year 2015

In [42]:
start_date = date(2015,6,1)
end_date = date(2015,12,30)

In [43]:
finance_pairs= [(x,y) for x in finance for y in finance if x!=y] #craeting combination of pairs so the are iterable 
finance_dict = {}

In [44]:
for x,y in finance_pairs:
    finance_dict[str(x) + str(y)] = rolling_adf(x,y,start_date,end_date)

In [45]:
finance_tradable_pairs = []
finance_tradable_stocks = []
for x,y in finance_pairs:
    mean = finance_dict[str(x) + str(y)][2]
    if mean <= -2.5:
        finance_tradable_pairs.append((x,y))
        finance_tradable_stocks.append(x)
        finance_tradable_stocks.append(y)
finance_tradable_pairs = set(tuple(sorted(p)) for p in finance_tradable_pairs)
        


In [46]:
print(set(finance_tradable_stocks))
print(finance_tradable_pairs)

{'DHFL', 'PFC', 'MANAPPURAM', 'L&TFH', 'HDFC'}
{('L&TFH', 'PFC'), ('HDFC', 'PFC'), ('DHFL', 'L&TFH'), ('MANAPPURAM', 'PFC'), ('HDFC', 'MANAPPURAM')}


In [225]:
dataframe(finance_dict,finance_tradable_pairs)

,x_stock,y_stock,rolling_adf_mean,rolling_beta_mean
0,L&TFH,PFC,-2.547787,3.527107
1,HDFC,PFC,-2.840804,0.190038
2,DHFL,L&TFH,-3.459738,0.199692
3,MANAPPURAM,PFC,-2.538808,9.536415
4,HDFC,MANAPPURAM,-2.690310,0.019890
